# 🧬 Psychedelic Therapeutics Dashboard

## Interactive analysis of psychedelic compounds with 2D analytics and 3D molecular visualization

This notebook demonstrates the complete pipeline for analyzing psychedelic compounds, computing molecular descriptors, and creating both 2D and 3D visualizations.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
src_path = Path('../src')
sys.path.insert(0, str(src_path.absolute()))

# Import our custom modules
from data import load_demo, validate_smiles_column
from descriptors import compute, get_descriptor_stats
from viz2d import plot_dashboard, plot_correlation_heatmap, plot_class_comparison
from viz3d import smiles_to_3d_viewer, smiles_to_molblock, batch_convert_to_sdf

print("✅ All imports successful!")

## 1. Data Loading and Validation

In [ ]:
# Load demo dataset
print("📊 Loading psychedelic compounds dataset...")
df = load_demo()

print(f"Loaded {len(df)} compounds")
print(f"Compound classes: {', '.join(df['class'].unique())}")

# Display the dataset
df

In [ ]:
# Validate SMILES strings
print("🔍 Validating SMILES strings...")
validate_smiles_column(df)
print("✅ All SMILES strings are valid!")

## 2. Molecular Descriptor Calculation

In [ ]:
# Compute molecular descriptors
print("⚗️ Computing molecular descriptors...")
df_with_descriptors = compute(df)

print("✅ Descriptors computed successfully!")

# Display results
display_cols = ['name', 'class', 'mw', 'logp', 'tpsa', 'hbd', 'hba', 'drug_likeness', 'bbb_label']
df_with_descriptors[display_cols]

In [ ]:
# Get summary statistics
stats = get_descriptor_stats(df_with_descriptors)

print("📊 Molecular Descriptor Summary:")
print(f"Molecular Weight: {stats['mw']['min']:.1f} - {stats['mw']['max']:.1f} Da (avg: {stats['mw']['mean']:.1f})")
print(f"LogP: {stats['logp']['min']:.2f} - {stats['logp']['max']:.2f} (avg: {stats['logp']['mean']:.2f})")
print(f"TPSA: {stats['tpsa']['min']:.1f} - {stats['tpsa']['max']:.1f} Ų (avg: {stats['tpsa']['mean']:.1f})")
print(f"Drug-likeness: {stats['drug_likeness']['min']:.2f} - {stats['drug_likeness']['max']:.2f} (avg: {stats['drug_likeness']['mean']:.2f})")

# BBB penetration distribution
bbb_counts = df_with_descriptors['bbb_label'].value_counts()
print(f"\n🧠 BBB Penetration Prediction:")
for label, count in bbb_counts.items():
    percentage = (count / len(df_with_descriptors)) * 100
    print(f"   {label}: {count} compounds ({percentage:.1f}%)")

## 3. 2D Visualization Dashboard

In [ ]:
# Generate main dashboard
print("📈 Creating 2D analysis dashboard...")
dashboard_path = plot_dashboard(df_with_descriptors)

# Display the dashboard in notebook
from IPython.display import Image, display
display(Image(dashboard_path))

In [ ]:
# Generate correlation heatmap
print("🔥 Creating correlation heatmap...")
heatmap_path = plot_correlation_heatmap(df_with_descriptors)

# Display heatmap
display(Image(heatmap_path))

In [ ]:
# Generate class comparison plots
print("📊 Creating class comparison plots...")
comparison_path = plot_class_comparison(df_with_descriptors)

# Display comparison plots
display(Image(comparison_path))

## 4. 3D Molecular Visualization

Now let's create interactive 3D visualizations of our psychedelic compounds.

In [ ]:
# Select a representative compound for 3D visualization
# Let's use 2C-B as our example
compound_name = "2C-B"
compound_data = df_with_descriptors[df_with_descriptors['name'] == compound_name].iloc[0]

print(f"🧬 Creating 3D visualization for {compound_name}")
print(f"SMILES: {compound_data['smiles']}")
print(f"Class: {compound_data['class']}")
print(f"MW: {compound_data['mw']:.1f} Da")
print(f"LogP: {compound_data['logp']:.2f}")
print(f"TPSA: {compound_data['tpsa']:.1f} Ų")
print(f"Drug-likeness: {compound_data['drug_likeness']:.2f}")
print(f"BBB: {compound_data['bbb_label']}")

In [ ]:
# Create 3D viewer for 2C-B
smiles_2cb = compound_data['smiles']

# Generate 3D structure
view_2cb = smiles_to_3d_viewer(smiles_2cb, width=800, height=600, style="stick")

if view_2cb is not None:
    print("✅ 3D structure generated successfully!")
    view_2cb.show()
else:
    print("❌ Failed to generate 3D structure")

In [ ]:
# Create 3D viewers for multiple compounds with different styles
compounds_to_show = ["2C-B", "2C-I", "DOB", "Mescaline"]
styles = ["stick", "sphere", "line", "stick"]

for compound_name, style in zip(compounds_to_show, styles):
    if compound_name in df_with_descriptors['name'].values:
        compound_row = df_with_descriptors[df_with_descriptors['name'] == compound_name].iloc[0]
        smiles = compound_row['smiles']
        
        print(f"\n🧬 {compound_name} ({style} style):")
        
        view = smiles_to_3d_viewer(smiles, width=600, height=400, style=style)
        if view is not None:
            view.show()
        else:
            print(f"❌ Failed to generate 3D structure for {compound_name}")

## 5. Export to SDF Format

In [ ]:
# Convert all compounds to 3D and export to SDF
print("💾 Converting all compounds to 3D and exporting to SDF...")

try:
    sdf_path = batch_convert_to_sdf(
        df_with_descriptors, 
        smiles_col='smiles', 
        name_col='name', 
        output_file='psychedelic_compounds_3d.sdf'
    )
    print(f"✅ SDF export successful: {sdf_path}")
    
    # Show file size
    from pathlib import Path
    sdf_file = Path(sdf_path)
    if sdf_file.exists():
        file_size = sdf_file.stat().st_size
        print(f"File size: {file_size:,} bytes")
        
except Exception as e:
    print(f"❌ SDF export failed: {str(e)}")

## 6. Compound Analysis Summary

In [ ]:
# Create comprehensive analysis summary
print("📋 PSYCHEDELIC THERAPEUTICS ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 Dataset Overview:")
print(f"   • Total compounds: {len(df_with_descriptors)}")
print(f"   • Compound classes: {df_with_descriptors['class'].nunique()}")
print(f"   • Classes: {', '.join(df_with_descriptors['class'].unique())}")

# Top compounds by drug-likeness
print(f"\n🏆 Top Compounds by Drug-likeness:")
top_compounds = df_with_descriptors.nlargest(5, 'drug_likeness')
for idx, (_, row) in enumerate(top_compounds.iterrows(), 1):
    print(f"   {idx}. {row['name']} ({row['class']}): {row['drug_likeness']:.2f}")

# CNS-favorable compounds
cns_favorable = df_with_descriptors[df_with_descriptors['bbb_label'] == 'Good BBB']
print(f"\n🧠 CNS-Favorable Compounds ({len(cns_favorable)}):")
for _, row in cns_favorable.iterrows():
    print(f"   • {row['name']}: TPSA = {row['tpsa']:.1f} Ų, Drug-likeness = {row['drug_likeness']:.2f}")

# Class-wise analysis
print(f"\n📈 Class-wise Analysis:")
class_summary = df_with_descriptors.groupby('class').agg({
    'mw': 'mean',
    'logp': 'mean', 
    'tpsa': 'mean',
    'drug_likeness': 'mean'
}).round(2)

for class_name, row in class_summary.iterrows():
    print(f"   • {class_name}:")
    print(f"     - Avg MW: {row['mw']:.1f} Da")
    print(f"     - Avg LogP: {row['logp']:.2f}")
    print(f"     - Avg TPSA: {row['tpsa']:.1f} Ų")
    print(f"     - Avg Drug-likeness: {row['drug_likeness']:.2f}")

print(f"\n🎯 Key Insights:")
print(f"   • Best overall compound: {top_compounds.iloc[0]['name']} (drug-likeness: {top_compounds.iloc[0]['drug_likeness']:.2f})")
print(f"   • Most CNS-favorable class: {cns_favorable['class'].mode().iloc[0] if len(cns_favorable) > 0 else 'None'}")
print(f"   • Average molecular weight: {df_with_descriptors['mw'].mean():.1f} Da")
print(f"   • BBB penetration success rate: {len(cns_favorable)/len(df_with_descriptors)*100:.1f}%")

print(f"\n✅ Analysis completed successfully!")

## 7. Interactive Exploration

Use the cells below to explore specific compounds or test custom SMILES strings.

In [ ]:
# Interactive compound explorer
# Change the compound name to explore different molecules
explore_compound = "2C-I"  # Try: "2C-B", "2C-I", "DOB", "DOM", "Mescaline", etc.

if explore_compound in df_with_descriptors['name'].values:
    compound = df_with_descriptors[df_with_descriptors['name'] == explore_compound].iloc[0]
    
    print(f"🔍 Exploring: {compound['name']}")
    print(f"Class: {compound['class']}")
    print(f"SMILES: {compound['smiles']}")
    print(f"Molecular Weight: {compound['mw']:.1f} Da")
    print(f"LogP: {compound['logp']:.2f}")
    print(f"TPSA: {compound['tpsa']:.1f} Ų")
    print(f"Drug-likeness: {compound['drug_likeness']:.2f}")
    print(f"BBB Penetration: {compound['bbb_label']}")
    
    # Create 3D viewer
    view = smiles_to_3d_viewer(compound['smiles'], width=700, height=500)
    if view:
        view.show()
else:
    print(f"❌ Compound '{explore_compound}' not found in dataset")
    print(f"Available compounds: {', '.join(df_with_descriptors['name'].tolist())}")

In [ ]:
# Test custom SMILES
# Enter your own SMILES string to analyze
custom_smiles = "CCc1cc(F)c(OCc2ccccc2)c(F)c1CCN"  # Example: 2C-B with fluorine instead of bromine

print(f"🧪 Analyzing custom SMILES: {custom_smiles}")

from descriptors import smiles_to_mol, calculate_descriptors, calculate_drug_likeness, bbb_label

# Validate and analyze
mol = smiles_to_mol(custom_smiles)
if mol is not None:
    descriptors = calculate_descriptors(mol)
    drug_likeness = calculate_drug_likeness(descriptors)
    bbb = bbb_label(descriptors['tpsa'])
    
    print(f"✅ Valid SMILES - Analysis completed:")
    print(f"   • Molecular Weight: {descriptors['mw']:.1f} Da")
    print(f"   • LogP: {descriptors['logp']:.2f}")
    print(f"   • TPSA: {descriptors['tpsa']:.1f} Ų")
    print(f"   • H-bond donors: {descriptors['hbd']:.0f}")
    print(f"   • H-bond acceptors: {descriptors['hba']:.0f}")
    print(f"   • Rotatable bonds: {descriptors['rotb']:.0f}")
    print(f"   • Drug-likeness: {drug_likeness:.2f}")
    print(f"   • BBB Penetration: {bbb}")
    
    # Create 3D viewer
    print(f"\n🧬 3D Structure:")
    view = smiles_to_3d_viewer(custom_smiles, width=600, height=400)
    if view:
        view.show()
    else:
        print("❌ Could not generate 3D structure")
        
else:
    print(f"❌ Invalid SMILES string: {custom_smiles}")

## Conclusion

This notebook demonstrates a complete pipeline for analyzing psychedelic compounds:

1. **Data Loading**: Load psychedelic compound datasets with SMILES structures
2. **Descriptor Calculation**: Compute molecular properties including drug-likeness and BBB penetration
3. **2D Visualization**: Create comprehensive analysis dashboards
4. **3D Visualization**: Generate interactive molecular viewers
5. **Export**: Save structures to SDF format for further analysis

### Key Features:
- ✅ **Comprehensive molecular analysis** with RDKit
- ✅ **Interactive 3D visualization** with py3Dmol
- ✅ **Drug-likeness assessment** based on Lipinski's rules
- ✅ **BBB penetration prediction** for CNS applications
- ✅ **Export capabilities** for computational chemistry workflows

### Next Steps:
- Run the Streamlit app: `streamlit run app/streamlit_app.py`
- Explore the command-line pipeline: `python -m src.pipeline`
- Customize with your own compound datasets
- Integrate with molecular docking workflows

🧬 **Happy molecular modeling!** 🚀